# snapReactors

Copyright (c) Dan Kotlyar and CoRE group

# Property Container

* This container stores all relevant info required for a specific 
property and provides methods to evaluate propertys based on specfic 
dependencies such as temperature, pressure, etc.
* The container also organizes propertys based on their type, providing subclasses such
as Constant, Table, and Correlation. 

### Code: 

```python
    >>> p2 = Table('h', 'THPHYS', np.array([1, 2, 3, 4]), 'W/K*m^2', 
    >>>     np.array([100, 200, 300, 400]), 'K', 
    >>>     unc = np.array([.01, .01, .01, .01]))
    >>>
    >>> p2.evaluate(300) #300K 
```

### Notes:

* This snippet shows the use of the tabulated property container ``Property.Table``.
* Property values can be given in tabulated fashion with up to two dependencies.
* The ``Property.evaluate(dependecys)`` method can be used to evaluate the method
for different dependency values.

In [1]:
import numpy as np
import sympy as sp
from snapReactors.containers.property import Property, Constant, Table, Correlation

#### Defining a constant property
1. Give name of property, must be in ``ALLOWED_PROPERTIES``
2. Give type of property, must be in ``Enum.PTYPE``
3. Give value of property
4. Give units of property
5. value uncertainty, ref, and description are left as optional parameters

In [2]:
p1 = Constant(id='cv', ptype='THPHYS', value=1, unit='kg', unc=None, ref=None, description=None)
print(p1)

{'id': 'cv', 'ptype': <PTYPE.THPHYS: 1>, 'dtype': <DTYPE.NUMBER: 1>, 'vtype': <VTYPE.CONSTANT: 1>, 'value': array([1]), 'valueUnit': 'kg', 'unc': None, 'dependents': None, 'dependentsUnit': None, 'description': None, 'ref': None}


#### Defining a tabulated property
1. Give name of property, must be in ``ALLOWED_PROPERTIES``
2. Give type of property, must be in ``Enum.PTYPES``
3. Give values of property
4. Give units of property
5. Give values of 1st dependency 
6. Give units of 1st dependency 
7. 2nd dependency values, 2nd dependency units, value uncertainty, reference, and description are left as optional parameters

In [3]:
p2 = Table('h', 'THPHYS', np.array([1, 2, 3, 4]), 'W/K*m^2', np.array([100, 200, 300, 400]), 'K',  unc = np.array([.01, .01, .01, .01]), dependency2=None, dependencyUnit2=None, ref=None, description=None)
print(p2)

{'id': 'h', 'ptype': <PTYPE.THPHYS: 1>, 'dtype': <DTYPE.NDARRAY: 2>, 'vtype': <VTYPE.TABLE: 2>, 'value': array([1, 2, 3, 4]), 'valueUnit': 'W/K*m^2', 'unc': array([0.01, 0.01, 0.01, 0.01]), 'dependents': array([100, 200, 300, 400]), 'dependentsUnit': 'K', 'description': None, 'ref': None, 'dependency1': array([100, 200, 300, 400]), 'dependency2': None, 'dependencyUnit1': 'K', 'dependencyUnit2': None}


#### Defining a correlation based property
1. Give name of property, must be in ``ALLOWED_PROPERTIES``
2. Give type of property, must be in ``Enum.PTYPES``
3. Give sympy expression for correlation
4. Give sympy symbols for expression
4. Give units of property
5. Give dependency range, bounds for the correlation
6. Give units of dependency 
7. value uncertainty, reference, and description are left as optional parameters

In [4]:
T = sp.symbols('T')
corr1 = T**2 + 1/2

p3 = Correlation('h', 'THPHYS', corr1, T, 'W/K*m^2', np.array([300, 600]), 'K', unc=None, ref=None, description=None)
print(p3)

{'id': 'h', 'ptype': <PTYPE.THPHYS: 1>, 'dtype': <DTYPE.NDARRAY: 2>, 'vtype': <VTYPE.CORRELATION: 3>, 'value': array([ 90000.5       ,  93711.45376926,  97497.37630154, 101358.26759683,
       105294.12765514, 109304.95647647, 113390.75406081, 117551.52040816,
       121787.25551853, 126097.95939192, 130483.63202832, 134944.27342774,
       139479.88359017, 144090.46251562, 148776.01020408, 153536.52665556,
       158372.01187005, 163282.46584756, 168267.88858809, 173328.28009163,
       178463.64035818, 183673.96938776, 188959.26718034, 194319.53373594,
       199754.76905456, 205264.97313619, 210850.14598084, 216510.2875885 ,
       222245.39795918, 228055.47709288, 233940.52498959, 239900.54164931,
       245935.52707205, 252045.48125781, 258230.40420658, 264490.29591837,
       270825.15639317, 277234.98563099, 283719.78363182, 290279.55039567,
       296914.28592253, 303623.99021241, 310408.66326531, 317268.30508122,
       324202.91566014, 331212.49500208, 338297.04310704, 345456

### Evaluating Propertys at specified dependencys
1. The ``Property.evaluate(dependencys)`` method is used to evaluate propertys at specfied dependencys
2. The evaluate method distinguish between propertys based on their type
3. The value returned by the method will be different evaluated differently for each type.

#### Evaluating Constants at specified dependencys
1. Constant will remain constant regardless of dependencies

In [5]:
v1p1 = p1.evaluate(300, 1500)
v2p1 = p1.evaluate(400, 1500)

print(v1p1, v2p1)




1.0 1.0


#### Evaluating Correlation propertys at specified dependencys
1. Correlations only support 1 dependency for now and will output the evualted value at the given dependency

In [6]:
v1p3 = p3.evaluate(dependency1=300)
v2p3 = p3.evaluate(dependency1=400)

print(v1p3, v2p3)

90000.5 160000.5


#### Evaluating Tabulated propertys at specified dependencys
1. Tabulated propertys only support 2 dependencys for now and will output the evualted value at the given dependencys based off a linear interpolation method


In [7]:
v1p2 = p2.evaluate(dependency1=300, dependency2=800)
v2p2 = p2.evaluate(dependency1=400, dependency2=1500)

print(v1p2, v2p2)

None None
